# Summary

The purpose of the this pipeline is to take in the illumina fastQ files (read 1 and read 2) and to output a text file containing the read number frequency of each unique viral barcode.  

## Input

read1.fastq, read2.fastq, reference.fasta

## Output

LinkGe csv(.txt) file containing  
- barcode sequences
- number of associated reads
- barcode frequency, expressed as a decimal 
- number of sequences passing cutoff after discarding barcodes containing "N"s
- frequency of barcode and wildtype reads 

## Pipeline steps

1. Map cleaned consensus.fasta to reference using bbmap, generates a .sam alignment file
2. Convert .sam to .bam alignment using reformat.sh
3. Generate a sorted .bam alignment using samtools
4. Run LinkGe.pl (perl script) to count number of reads per unique barcode, writes to csv file
5. Discard viral barcodes containing an "N", degenerate nucleotide 
6. Determine the number of sequences passing cutoff after discarding seqs containing Ns --> modified conversion rate 
7. Extract wildtype and barcode frequency from the LinkGe.csv output file

**File naming system:**

**First Digit**

Groups | Mixture composition 
--------|--------------------
1 | 100% bc
2 | 90% bc
3 | 50% bc
4 | 10% bc
5 | 100% wt


**Second Digit**

Out | Dilutions, input copy numbers
----|------------------------------
1 | 100,000
2 | 10,000
3 | 1,000
4 | 100 

**Examples**
- 1,1: 100% barcoded mixture, 100,000 input copies 
- 3,2: 50% barcoded mixture, 10,000 input copies
- 5,4: 100% wildtype mixture, 100 input copies 

In [7]:
%%bash

# pair and merge raw fastQ files, read 1 and read 2
# output 1: zipped paired-merged.fastq
# output 2: zipped paired-unmerged.fastq
# output 3: histogram with insert sizes and read counts

in1="ZIKV_trad_rep1_1,1_R1.fastq.gz"
in2="ZIKV_trad_rep1_1,1_R2.fastq.gz"
out="out_1/ZIKV_rep1_1,1.fastq"
outu="out_1/ZIKV_rep1_1,1-unmerged.fastq"
ihist="out_1/ZIKV_rep1_1,1-hist.txt"

/Users/katbraun/anaconda3/bin/bbmerge.sh \
in1=$in1 \
in2=$in2 \
out=$out \
outu=$outu \
ihist=$ihist



java -Djava.library.path=/Users/katbraun/anaconda3/opt/bbmap-38.22-0/jni/ -ea -Xmx1000m -Xms1000m -cp /Users/katbraun/anaconda3/opt/bbmap-38.22-0/current/ jgi.BBMerge in1=ZIKV_trad_rep1_1,1_R1.fastq.gz in2=ZIKV_trad_rep1_1,1_R2.fastq.gz out=out_1/ZIKV_rep1_1,1.fastq outu=out_1/ZIKV_rep1_1,1-unmerged.fastq ihist=out_1/ZIKV_rep1_1,1-hist.txt
Executing jgi.BBMerge [in1=ZIKV_trad_rep1_1,1_R1.fastq.gz, in2=ZIKV_trad_rep1_1,1_R2.fastq.gz, out=out_1/ZIKV_rep1_1,1.fastq, outu=out_1/ZIKV_rep1_1,1-unmerged.fastq, ihist=out_1/ZIKV_rep1_1,1-hist.txt]
Version 38.22

Writing mergable reads merged.
Started output threads.
Total time: 17.957 seconds.

Pairs:               	500707
Joined:              	466504   	93.169%
Ambiguous:           	28000   	5.592%
No Solution:         	6203       	1.239%
Too Short:           	0       	0.000%

Avg Insert:          	153.2
Standard Deviation:  	70.2
Mode:                	131

Insert range:        	39 - 493
90th percentile:     	132
75th percentile:     	131
50th

In [8]:
%%bash

# map consensus.fasta to reference 
# output 1: alignment in .sam format
# output 2: base histogram (bhist) lists base frequencies at each position
# output 3: quality histogram (qhist) by position 
# output 4: histogram (aqhist) of average read quality 
# output 5: histogram (lhist) of read lengths

file="out_1/ZIKV_rep1_1,1.fastq"
ref="../ZIKV_ref.fasta"
out="out_1/paired-merged-aligned-1,1.sam"

/Users/katbraun/anaconda3/bin/bbmap.sh \
ref=$ref \
in=$file \
out=$out \
# bhist=LinkGe/base-histogram.txt \
# qhist=LinkGe/quality-histogram.txt \
# aqhist=LinkGe/average-quality-histogram.txt \
# lhist=LinkGe/lengths-histogram.txt \
printunmappedcount=t \
mintrimlength=131 \ 
statsfile=stderr

Max memory cannot be determined.  Attempting to use 3200 MB.
If this fails, please add the -Xmx flag (e.g. -Xmx24g) to your command, 
or run this program qsubbed or from a qlogin session on Genepool, or set ulimit to an appropriate value.
java -Djava.library.path=/Users/katbraun/anaconda3/opt/bbmap-38.22-0/jni/ -ea -Xmx3200m -cp /Users/katbraun/anaconda3/opt/bbmap-38.22-0/current/ align2.BBMap build=1 overwrite=true fastareadlen=500 ref=../ZIKV_ref.fasta in=out_1/ZIKV_rep1_1,1.fastq out=out_1/paired-merged-aligned-1,1.sam
Executing align2.BBMap [build=1, overwrite=true, fastareadlen=500, ref=../ZIKV_ref.fasta, in=out_1/ZIKV_rep1_1,1.fastq, out=out_1/paired-merged-aligned-1,1.sam]
Version 38.22

Retaining first best site only for ambiguous mappings.
NOTE:	Ignoring reference file because it already appears to have been processed.
NOTE:	If you wish to regenerate the index, please manually delete ref/genome/1/summary.txt
Set genome to 1

Loaded Reference:	0.071 seconds.
Loading index for c

In [9]:
%%bash

# convert sam alignment to bam alignment 
# 'sam=1.3' flag is required to output a legacy bam file that will be compatible with LinkGe
# output 1: paired-merged-aligned.bam 

file="out_1/ZIKV_rep1_1,1.fastq"
inpath="out_1/paired-merged-aligned-1,1.sam"
outpath="out_1/paired-merged-aligned-1,1.bam"

/Users/katbraun/anaconda3/bin/reformat.sh \
'sam=1.3' \
in=$inpath \
out=$outpath

java -ea -Xmx200m -cp /Users/katbraun/anaconda3/opt/bbmap-38.22-0/current/ jgi.ReformatReads sam=1.3 in=out_1/paired-merged-aligned-1,1.sam out=out_1/paired-merged-aligned-1,1.bam
Executing jgi.ReformatReads [sam=1.3, in=out_1/paired-merged-aligned-1,1.sam, out=out_1/paired-merged-aligned-1,1.bam]

Input is being processed as unpaired
Found samtools 1.9
Input:                  	466504 reads          	71469859 bases
Output:                 	466504 reads (100.00%) 	71469859 bases (100.00%)

Time:                         	3.220 seconds.
Reads Processed:        466k 	144.86k reads/sec
Bases Processed:      71469k 	22.19m bases/sec


In [10]:
%%bash

# sort the bam alignment 
# output 1: paired-merged-aligned-sorted.bam 

file="out_1/ZIKV_rep1_1,1.fastq"
inpath="out_1/paired-merged-aligned-1,1.bam"
outpath="out_1/paired-merged-aligned-sorted-1,1.bam"

/Users/katbraun/anaconda3/bin/samtools \
sort -o $outpath $inpath

In [11]:
%%bash

# run LinkGe on paired-merged-aligned-sorted.bam at
# barcode positions: 60, 63, 66, 69, 72, 75, 78, 81
# output 1: csv with barcode sequence, reads per barcodes sequence, and frequency 

file="out_1/ZIKV_rep1_1,1.fastq"
outfile="LinkGe-rep1-1,1"
inpath="out_1/paired-merged-aligned-sorted-1,1.bam"
Linkge="../scripts/LinkGe.pl"

mkdir out_1/LinkGe

/Users/katbraun/anaconda3/bin/perl \
$Linkge -o $outfile -l 60,63,66,69,72,75,78,81 $inpath
mv $outfile.txt out_1/LinkGe/$outfile.txt

parsing position 60 ... 420042 reads
parsing position 63 ... 420042 reads
parsing position 66 ... 420042 reads
parsing position 69 ... 420042 reads
parsing position 72 ... 420042 reads
parsing position 75 ... 420041 reads
parsing position 78 ... 420039 reads
parsing position 81 ... 420039 reads
finding reads that cover all positions ... 420039 matches
counting reads with matching calls ... 
printing LinkGe-rep1-1,1.txt to /Users/katbraun/Documents/research/kat_braun/projects/UMI_method/data_derived/run_476/traditional_amplicons_rep1/group_1 ... 
script completed in 49s


In [12]:
%%bash

# determine the number of lines in the LinkGe.txt file expected 
# after the lines containing an "N" are removed 

file="out_1/LinkGe/LinkGe-rep1-1,1.txt"
echo $path

Tlines=$(wc -l $file | cut -f1 -d' ') # total number of lines in the fasta file
Rlines=$(echo "$(($Tlines - 3))") # number of lines discarding 3 header lines 
Nlines=$(grep "N" $file | wc -l) #  lines containing at least one "N"
lines=$(echo "$(($Rlines - $Nlines))") # lines expected after degenerate barcodes are discarded

# echo $Tlines
# echo $Rlines
echo $Nlines
echo $lines


5535
42084


In [13]:
%%bash 

# discard all lines in LinkGe.txt that contain ambiguous bases (Ns)
# this will fail if there is an N in the UMI (fasta header), the next cell is a check for this potential problem 

file="out_1/LinkGe/LinkGe-rep1-1,1.txt"

sed -i '/N/d' $file

In [14]:
%%bash

# check to ensure that the above sed command worked as expected
# $lines (from above) and $lines (from this cell) should be equal, if not an error occured

file="out_1/LinkGe/LinkGe-rep1-1,1.txt"

lines=$(wc -l $file | cut -f1 -d' ')
Tlines=$(echo "$(($lines - 3))") # number of lines discarding 3 header lines 
echo $Tlines

42084


In [15]:
%%bash

# this cell determines the number of sequences passing cutoff, discarding sequencing containing "Ns"


file="out_1/LinkGe/LinkGe-rep1-1,1.txt"
csv="out_1/LinkGe/LinkGe-rep1-1,1.csv"

cut -f2 -d$'\t' $file >> $csv
sed -i '/'LinkGe'/d' $csv 
sed -i '/'Total'/d' $csv
sed -i '/'reads'/d' $csv

sum=$(awk -F',' '{sum+=$1} END {print sum}' $csv)
echo "Number of sequences passing cutoff after discarding seqs containing Ns:" $sum

Number of sequences passing cutoff after discarding seqs containing Ns: 413593


In [16]:
%%bash

# extract wildtype freqency from the LinkGe.txt file 

path="out_1/LinkGe/LinkGe-rep1-1,1.txt"

wt=$(grep "G60:T63:T66:G69:A72:A75:G78:C81" $path | cut -f3 -d$'\t') # wildtype frequency, decimal form

echo $wt


0.0096372


In [17]:
# this is a python cell because bash is bad at math
# copy/paste $wt from above into the wt = $wt line 
# outputs wildtype and barcode frequency in percentage form 

wt = 0.0096372 # use the out from above cell 
wtfreq = wt * 100
bcfreq = 100 - wtfreq
totalfreq = wtfreq + bcfreq

print('Wildtype frequency is:', wtfreq)
print('Barcode frequency is:', bcfreq)
     
print('Check to make sure wt + bc frequency equals one-hundred:',totalfreq)

Wildtype frequency is: 0.96372
Barcode frequency is: 99.03628
Check to make sure wt + bc frequency equals one-hundred: 100.0
